# Spatial Analysis and Modeling

### Instructor: Rossano Schifanella
Email: [rossano.schifanella@unito.it](mailto:rossano.schifanella@unito.it)

## Represent Spatial Data

## Exercise 1

In this exercise you will learn how to:

* Create a spatial database
* Create a table with spatial columns 
* Create indexes on spatial columns
* Load data from a 
    * ESRI Shapefile
    * OpenStreetMap metro extract
    * CSV
* Transform SRID of a geometry column 

Due to the limited time of the practical session, after reading this note you should run the bash script in **scripts/1-process/build_env.sh** to build the environment (i.e., database, tables, load the data) that we will use throughout the tutorial. 


## Create a spatial database

Connect to the interactive terminal for working with Postgres

> psql template1

Create the database _geocycle_ we will use throughout the tutorial

> CREATE DATABASE geocycle; 

Connect to the database _geocycle_ we just created

> \\connect geocycle;

Enable PostGIS through the PostgreSQL extensions mechanism.

**CREATE EXTENSION** loads a new extension into the current database. 

Loading an extension essentially amounts to running the extension's script file. The script will typically create new SQL objects such as functions, data types, operators and index support methods. **CREATE EXTENSION** additionally records the identities of all the created objects, so that they can be dropped again if **DROP EXTENSION** is issued.

> CREATE EXTENSION POSTGIS;

> CREATE EXTENSION HSTORE;

## Create a table with a spatial column

Create the table *instagram\_photos* that contains a set of geotagged photos in New York City

> CREATE TABLE IF NOT EXISTS instagram_photos ( <br/>
> pid text NOT NULL,<br/>
> username text, <br/>
> created_at timestamp,<br/>
> longitude float8, <br/>
> latitude float8, <br/>
> caption text,<br/>
> tags text array);

Add a geometry column _geom_ as a two dimensional point

> SELECT AddGeometryColumn('instagram_photos', 'geom', 4326, 'POINT', 2);

Convert _longitude_ and _latitude_ columns in a **Point** geometry

> UPDATE instagram_photos SET geom = ST_SetSRID(ST_Point(longitude, latitude), 4326);

Otherwise it's possible to generate the spatial column in a single step

> CREATE TABLE IF NOT EXISTS instagram_photos ( <br/>
>    pid bigint NOT NULL, <br/>
>    username text, <br/>
>    created_at timestamp,<br/>
>    geom geometry(Point, 4326), <br/>
>    caption text,<br/>
>    tags text array); <br/>

Create the table *nyc\_crime* that contains the information about (a sample) of the criminal activity in New Yor City during 2015:

> CREATE TABLE IF NOT EXISTS nyc_crime ( <br/>
> identifier text NOT NULL,<br/>
> date timestamp, <br/>
> offence text, <br/>
> category text, <br/>
> longitude float8, <br/>
> latitude float8);

> SELECT AddGeometryColumn('nyc_crime', 'geom', 4326, 'POINT', 2);

> UPDATE nyc_crime SET geom = ST_SetSRID(ST_Point(longitude, latitude), 4326);


## Create indexes on a spatial column

Add GIST index to a geometry column

> CREATE INDEX geom_index on instagram_photos USING GIST(geom);

Add generic indexes to other columns if need be

> CREATE INDEX pid_index on instagram_photos(pid);

Indexes have to be created with the same syntax for the *nyc\_crime* table.


## Load data 

### From a ESRI shapefile

Load NYC boundary data in dfferent spatial aggregation:
* Boroughs
* District Communities
* Census Tracts
* Census Blocks

ESRI shapefile are loaded using the tool **shp2pgsql** ([tutorial](http://www.bostongis.com/pgsql2shp_shp2pgsql_quickguide_20.bqg))


> shp2pgsql -I -d -s 102718:4326 -g geom ../data/shp/nyc_boroughs/nyc_boroughs.shp nyc_boroughs | psql -d geocycle -U geocycle

![NYC Boroughs](images/nyc_boroughs.png)

> shp2pgsql -I -d -s 26918:4326 -g geom ../data/shp/nyc_neighborhoods/nyc_neighborhoods.shp nyc_neighborhoods | psql -d geocycle -U geocycle

![NYC Neighborhoods](images/nyc_neighborhoods.png)

> shp2pgsql -I -d -s 102718:4326 -g geom ../data/shp/nyc_census_tracts/nyc_census_tracts.shp nyc_census_tracts | psql -d geocycle -U geocycle

![NYC Census Tracts](images/nyc_census_tracts.png)

> shp2pgsql -I -d -s 102718:4326 -g geom ../data/shp/nyc_census_blocks/nyc_census_blocks.shp nyc_census_blocks | psql -d geocycle -U geocycle

![NYC Census Blocks](images/nyc_census_blocks.png)

### From a OpenStreetMap metro extract

There are different alternatives to load OpenStreetMap data into PostGIS:
* **osmosis** (**[link](http://wiki.openstreetmap.org/wiki/Osmosis)**)
* **osm2pgsql** (**[link](http://wiki.openstreetmap.org/wiki/Osm2pgsql)**)
* **imposm** (**[link](http://imposm.org/)**)

In this tutorial we will use **osmosis** that creates a schema that is easier to integrate in our tutorial.

Create the (default) schemas

> psql -d geocycle -U geocycle -f /usr/share/doc/osmosis/examples/pgsnapshot_schema_0.6.sql

> psql -d geocycle -U geocycle -f /usr/share/doc/osmosis/examples/pgsnapshot_schema_0.6_linestring.sql

Load OpenStreetMap data into the database

> osmosis --read-pbf ../data/osm/nyc_poly_highways.osm.pbf --log-progress --write-pgsql database=geocycle user=geocycle password=geocycle

![NYC OSM Ways](images/nyc_ways.png)

### From a CSV

Load Instagram photos from a comma separated file. Note that the input file path has to be absolute and that the number of columns in the target table and the features in the CSV file have to perfectly match.

> COPY instagram_photos FROM '/Users/schifane/Google_UNITO/talks/icwsm16-tutorial/data/instagram/instagram-photos-manhattan.csv' CSV HEADER ENCODING 'utf-8';

Load the New York City crime dataset

> COPY nyc_crime FROM '/Users/schifane/Google_UNITO/talks/icwsm16-tutorial/data/crime/nyc_crime.csv' CSV HEADER ENCODING 'utf-8';


## Transform SRID

Transform the geometry field _geom_ to SRID 4326. Note that you should replace _Point_ with the actual geometry type of your geometry column. 

> ALTER TABLE flickr_photos <br>
> ALTER COLUMN geom <br>
> TYPE Geometry(Point, 4326) <br>
> USING ST_Transform(geom, 4326); <br>	


